In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Python/dacon/raw

In [ ]:
!pip install autogluon

### 1. Load Data

In [ ]:
# !pip install blpapi --index-url=https://bcms.bloomberg.com/pip/simple/
# !pip install xbbg

# from xbbg import blp

# blp.bdp(tickers='BDIY IND', flds=['last_price'], start_date='2014-10-10', end_date='2023-10-18')

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
from autogluon.tabular import TabularDataset, TabularPredictor

# 로그 스케일링 함수
def log_scale(data, base=np.e):
    return np.log(data + 1e-9) / np.log(base)

def inverse_log_scale(log_data, base=np.e):
    """
    Convert log scaled data back to its original scale.

    Parameters:
    - log_data: The log scaled data.
    - base: The base of the logarithm. Default is the natural logarithm.

    Returns:
    - Data in its original scale.
    """

    return np.power(base, log_data) - 1e-9

# def cyclic_transform(data, col, max_val):
#     data['sin_' + col] = np.sin(2 * np.pi * data[col] / max_val)
#     data['cos_' + col] = np.sin(2 * np.pi * data[col] / max_val)

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### 2. Generate DateTime feature

In [ ]:
# train_df['time'] = pd.to_datetime(train_df['ATA'])
# train_df['year'] = train_df['time'].dt.year
# train_df['month'] = train_df['time'].dt.month
# train_df['day'] = train_df['time'].dt.day
# train_df['day_of_week'] = train_df['time'].dt.dayofweek

# test_df['time'] = pd.to_datetime(test_df['ATA'])
# test_df['year'] = test_df['time'].dt.year
# test_df['month'] = test_df['time'].dt.month
# test_df['day'] = test_df['time'].dt.day
# test_df['day_of_week'] = test_df['time'].dt.dayofweek

### + Imputing

In [ ]:
# missing_rows_train = train_df[train_df.isnull().any(axis=1)]
# missing_rows_test = test_df[test_df.isnull().any(axis=1)]

# group = train_df.groupby(by = ['ARI_CO', 'month']).mean().loc[:,['U_WIND','V_WIND','AIR_TEMPERATURE','BN']]

# #missing_rows['U_WIND'] = missing_rows.apply(lambda x: group.loc[x['ARI_CO'],x['month']]['U_WIND'], axis=1)
# #missing_rows['V_WIND'] = missing_rows.apply(lambda x: group.loc[x['ARI_CO'],x['month']]['V_WIND'], axis=1)
# missing_rows_train['AIR_TEMPERATURE'] = missing_rows_train.apply(lambda x: group.loc[x['ARI_CO'],x['month']]['AIR_TEMPERATURE'], axis=1)
# missing_rows_test['AIR_TEMPERATURE'] = missing_rows_test.apply(lambda x: group.loc[x['ARI_CO'],x['month']]['AIR_TEMPERATURE'], axis=1)
# #missing_rows['BN'] = missing_rows.apply(lambda x: group.loc[x['ARI_CO'],x['month']]['BN'], axis=1)

### 3. Datetime to Cyclic

In [ ]:
# cyclic_transform(data=train_df, col='ATA_LT', max_val=24)
# cyclic_transform(data=train_df, col='month', max_val=12)
# cyclic_transform(data=train_df, col='day', max_val=31)

# cyclic_transform(data=test_df, col='ATA_LT', max_val=24)
# cyclic_transform(data=test_df, col='month', max_val=12)
# cyclic_transform(data=test_df, col='day', max_val=31)

# train_df = train_df.drop(['ATA_LT', 'time', 'month', 'day'], axis=1)
# test_df = test_df.drop(['ATA_LT', 'time', 'month', 'day'], axis=1)

### 4. Merge oil & ship index

In [ ]:
train_df['ATA_date'] = pd.to_datetime(train_df['ATA']).map(lambda x: x.date())
test_df['ATA_date'] = pd.to_datetime(test_df['ATA']).map(lambda x: x.date())

oil = pd.read_csv('oil/oil_data.csv', index_col = 0)
oil['날짜'] = pd.to_datetime(oil['날짜']).map(lambda x: x.date())

ship_idx = pd.read_csv('oil/ship_idx_data.csv', index_col = 0)
ship_idx['날짜'] = pd.to_datetime(ship_idx['날짜']).map(lambda x: x.date())
ship_idx = ship_idx.drop_duplicates(subset='날짜', keep = 'first')

train_df = pd.merge(train_df, oil, left_on = "ATA_date", right_on ="날짜", how= 'left')
test_df = pd.merge(test_df, oil, left_on = "ATA_date", right_on ="날짜", how= 'left')

# 해상운임지수
train_df = train_df.drop(['날짜'], axis=1)
test_df = test_df.drop(['날짜'], axis=1)

train_df = pd.merge(train_df, ship_idx, left_on = "ATA_date", right_on ="날짜", how= 'left')
test_df = pd.merge(test_df, ship_idx, left_on = "ATA_date", right_on ="날짜", how= 'left')

train_df = train_df.drop(['ATA', '날짜'], axis=1)
test_df = test_df.drop(['ATA', '날짜'], axis=1)

In [ ]:
import numpy as np

for df in [train_df, test_df]:
  df["Volume"] = df["LENGTH"] * df["BREADTH"] * df["DEPTH"]
  #df["Wind_Speed"] = np.sqrt(df["U_WIND"]**2 + df["V_WIND"]**2)
  #df["Wind_Direction"] = np.arctan2(df["V_WIND"], df["U_WIND"]) * (180 / np.pi)  # Radian to Degree
  df["Load_Capacity"] = df["DEADWEIGHT"] / df["GT"]

In [ ]:
# IS_CI 컬럼 추가
train_df['IS_CI'] = (train_df['CI_HOUR'] != 0).astype(int)

In [ ]:
# 예시 코드 (실제 데이터에 적용하기 전에 확인)
from sklearn.preprocessing import StandardScaler

train_df['CI_HOUR'] = train_df['CI_HOUR'].map(log_scale)

# 표준화
scaler = StandardScaler()
train_df['CI_HOUR'] = scaler.fit_transform(train_df['CI_HOUR'].values.reshape(-1, 1))

for i in ["ARI_CO", "ARI_PO", "FLAG", "SHIPMANAGER"]:
    # 각 범주에 대한 CI_HOUR의 평균과 분산 계산
    grouped = train_df[train_df['CI_HOUR'] != 0].copy().groupby(i)['CI_HOUR'].agg(['mean', 'var'])
    grouped.columns = [i+'_mean', i+'_var']

    # 딕셔너리로 저장
    mean_dict = grouped[i+'_mean'].to_dict()
    var_dict = grouped[i+'_var'].to_dict()

    # 딕셔너리를 사용하여 train_df에 매핑
    train_df[i+'_mean'] = train_df[i].map(mean_dict)
    train_df[i+'_var'] = train_df[i].map(var_dict)

    # 딕셔너리를 사용하여 test_df에 매핑
    test_df[i+'_mean'] = test_df[i].map(mean_dict)
    test_df[i+'_var'] = test_df[i].map(var_dict)

# 결과를 확인하기 위한 코드 (실제 실행에는 필요하지 않음)
train_df.head()

In [ ]:
# # 'U_WIND', 'V_WIND', 'BN' 열에서 하나라도 결측치가 있는 행의 인덱스를 찾기
# missing_train = train_df[train_df[['U_WIND', 'V_WIND', 'BN']].isnull().any(axis=1)].index
# missing_test = test_df[test_df[['U_WIND', 'V_WIND', 'BN']].isnull().any(axis=1)].index

# train_df_1 = train_df.iloc[~train_df.index.isin(missing_train)]
# train_df_2 = train_df.iloc[missing_train]

# test_df_1 = test_df.iloc[~test_df.index.isin(missing_test)]
# test_df_2 = test_df.iloc[missing_test]

# print("True or False if train_df_1 plus 2 equals train_df: " + str(len(train_df) == len(train_df_1) +len(train_df_2)))
# print("True or False if test_df_1 plus 2 equals test_df: " + str(len(test_df) == len(test_df_1) +len(test_df_2)))

# # autogluon 학습을 위한 데이터 형태로 변환
# train_1 = TabularDataset(train_df_1.drop(['SAMPLE_ID','CI_HOUR', 'ID'], axis=1))
# test_1 = TabularDataset(test_df_1.drop(['SAMPLE_ID', 'ID'], axis=1))

# # autogluon 학습을 위한 데이터 형태로 변환
# train_2 = TabularDataset(train_df_2.drop(['SAMPLE_ID','CI_HOUR','U_WIND', 'V_WIND', 'BN', 'ID'], axis=1))
# test_2 = TabularDataset(test_df_2.drop(['SAMPLE_ID','U_WIND', 'V_WIND', 'BN','ID'], axis=1))

# print("True or False if train_df_1 plus 2 equals train_df: " + str(len(train_df) == len(train_1) +len(train_2)))
# print("True or False if test_df_1 plus 2 equals test_df: " + str(len(test_df) == len(test_1) +len(test_2)))

In [ ]:
train = TabularDataset(train_df.drop(['SAMPLE_ID','ARI_CO','ARI_PO','CI_HOUR', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE','BN', 'ID', 'SHIPMANAGER', 'FLAG'], axis=1))
train.columns

In [ ]:
train[train['IS_CI']==0]

In [ ]:
# # 이렇게 한 줄만 작성하면 내부에서 알아서 학습해줍니다.
# predictor_clf_1 = TabularPredictor(label='IS_CI', eval_metric='f1').fit(train_1)
# predictor_clf_2 = TabularPredictor(label='IS_CI', eval_metric='f1').fit(train_2)
#predictor_clf = TabularPredictor(label='IS_CI', eval_metric='f1').fit(train, hyperparameters ={'XGB': {}})
predictor_clf = TabularPredictor.load("AutogluonModels/ag-20231028_143508/")
#predictor_clf = TabularPredictor.load("AutogluonModels/ag-20231023_121256/")

In [ ]:
# 보류
# # IS_CI 컬럼 추가
# train_df['OVER_100'] = (train_df['CI_HOUR'] > 100).astype(int)

# train = TabularDataset(train_df[train_df['IS_CI'] == 1].drop(['SAMPLE_ID','CI_HOUR', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE','BN', 'ID', 'SHIPMANAGER', 'FLAG', 'IS_CI'], axis=1))
# train.columns

In [ ]:
train_df_reg = train_df[train_df['IS_CI'] == 1]

In [ ]:
# predictor_clf_2 = TabularPredictor(label='OVER_100', eval_metric='recall').fit(train)
# #predictor_clf_2 = TabularPredictor.load("AutogluonModels/ag-20231024_040233/") #f1

In [ ]:
# GOAT
# predictor_clf_1 = TabularPredictor.load("AutogluonModels/ag-20231022_082356/")
# predictor_clf_2 = TabularPredictor.load("AutogluonModels/ag-20231022_085339/")

# predictor = TabularPredictor.load("AutogluonModels/ag-20231022_113921/")
# predictor = TabularPredictor.load("AutogluonModels/ag-20231022_133024/")

In [ ]:
# # autogluon 학습을 위한 데이터 형태로 변환
# train_1 = TabularDataset(train_df_1.drop(['SAMPLE_ID','CI_HOUR', 'ID'], axis=1))
# test_1 = TabularDataset(test_df_1.drop(['SAMPLE_ID', 'ID'], axis=1))

# # autogluon 학습을 위한 데이터 형태로 변환
# train_2 = TabularDataset(train_df_2.drop(['SAMPLE_ID','CI_HOUR','U_WIND', 'V_WIND', 'BN', 'ID'], axis=1))
# test_2 = TabularDataset(test_df_2.drop(['SAMPLE_ID','U_WIND', 'V_WIND', 'BN','ID'], axis=1))

In [ ]:
# # autogluon regression 학습을 위한 데이터 형태로 변환
# train_1 = TabularDataset(train_df[train_df['IS_CI'] == 1].drop(['SAMPLE_ID','IS_CI', 'ID'], axis=1))
# # autogluon regression 학습을 위한 데이터 형태로 변환
# train_2 = TabularDataset(train_df[train_df['IS_CI'] == 1].drop(['SAMPLE_ID','IS_CI','U_WIND', 'V_WIND', 'BN', 'ID'], axis=1))

In [ ]:
train = TabularDataset(train_df_reg.drop(['SAMPLE_ID','ARI_CO','ARI_PO','IS_CI', 'U_WIND', 'V_WIND', 'AIR_TEMPERATURE','BN', 'ID', 'SHIPMANAGER', 'FLAG'], axis=1))
train.columns

In [ ]:
train

In [ ]:
# # 이렇게 한 줄만 작성하면 내부에서 알아서 학습해줍니다.
# predictor_clf_1 = TabularPredictor(label='IS_CI', eval_metric='f1').fit(train_1)
# predictor_clf_2 = TabularPredictor(label='IS_CI', eval_metric='f1').fit(train_2)

In [ ]:
hyperparam = {
    'NN_TORCH': {},
    'GBM': [
        {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
        {},
        'GBMLarge'
    ],
    #'CAT': {},
    'XGB': {},
    'FASTAI': {},
    'RF': [
        {'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
        {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
        {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}
    ],
    'XT': [
        {'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}},
        {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}},
        {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression']}}
    ],
    'KNN': [
        {'weights': 'uniform', 'ag_args': {'name_suffix': 'Unif'}},
        {'weights': 'distance', 'ag_args': {'name_suffix': 'Dist'}}
    ],
    #'FT_TRANSFORMER': {}
}

In [ ]:
# 이렇게 한 줄만 작성하면 내부에서 알아서 학습해줍니다.
predictor_reg = TabularPredictor(label='CI_HOUR', eval_metric='mae').fit(train, presets = "high_quality", hyperparameters = hyperparam)
# predictor_reg = TabularPredictor.load("AutogluonModels/ag-20231028_144913/") 현재 최종 모델

In [ ]:
# ld_board = predictor_reg.leaderboard(train, silent=True)
# ld_board.to_excel("ld_board.xlsx")

In [ ]:
test = TabularDataset(test_df.drop(['SAMPLE_ID','ARI_CO','ARI_PO', 'U_WIND', 'V_WIND', 'BN', 'ID'], axis=1))
pred_class = predictor_clf.predict(test)
pred_hour = predictor_reg.predict(test)

In [ ]:
# print(len(train['CLASS']) == len(predictor_clf.predict(train)))
# print(len(train['HOUR']) == len(predictor_reg.predict(train)))
# print(len(train['CI_HOUR']) == len(train_df[train_df['IS_CI'] == 1]['CI_HOUR']))

In [ ]:
# train['CLASS'] = predictor_clf.predict(train)
# train['HOUR'] = predictor_reg.predict(train)
# train['CI_HOUR'] = train_df[train_df['IS_CI'] == 1]['CI_HOUR']

# train['HOUR'] = scaler.inverse_transform(train['HOUR'].values.reshape(-1,1))
# train['HOUR'] = train['HOUR'].map(inverse_log_scale)
# train['CI_HOUR'] = scaler.inverse_transform(train['CI_HOUR'].values.reshape(-1,1))
# train['CI_HOUR'] = train['CI_HOUR'].map(inverse_log_scale)

In [ ]:
# train[['HOUR','CI_HOUR']].sort_values('CI_HOUR', ascending=False).tail(60)

In [ ]:
# train[['HOUR','CI_HOUR']].describe()

In [ ]:
# pred_class_1 = predictor_clf_1.predict(test_1)
# pred_class_2 = predictor_clf_2.predict(test_2)
# pred_hour_1 = predictor_reg_1.predict(test_1)
# pred_hour_2 = predictor_reg_2.predict(test_2)

In [ ]:
# pred_class = pred_class_1.append(pred_class_2).sort_index()
# pred_hour = pred_hour_1.append(pred_hour_2).sort_index()

In [ ]:
# 제출 파일 생성
submit = pd.DataFrame()

submit['SAMPLE_ID'] = test_df['SAMPLE_ID']
submit['CI_HOUR'] = scaler.inverse_transform(pred_hour.values.reshape(-1,1))
submit['CI_HOUR'] = submit['CI_HOUR'].map(inverse_log_scale)
submit['IS_CI'] = pred_class

# IS_CI 값이 0이면 CI_HOUR 값을 0으로 바꾸기
submit.loc[submit['IS_CI'] == 0, 'CI_HOUR'] = 0
submit.loc[submit['CI_HOUR'] < 0, 'CI_HOUR'] = 0

In [ ]:
submit = submit.drop(['IS_CI'], axis=1)
submit

In [ ]:
submit.to_csv(str(datetime.now())+ 'mae_submit.csv', index=False)

In [ ]:
submit.describe()

In [ ]:
submit.describe()